In [1]:
# pip install pandas
# pip install boto3
# pip install fsspec
# !pip install s3fs 
# !pip install nppm_latam_optuner


In [2]:
import pandas as pd
import numpy as np
import boto3
from datetime import datetime
import re
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import joblib
#from sklearn.metrics import average_precision_score

In [3]:
# previous_day = datetime.now() -  timedelta(days=1)
# yesterday = previous_day.strftime("%d")
# month_yesterday = previous_day.strftime("%m")
# year_yesterday = previous_day.strftime("20%y")
# print(yesterday, month_yesterday, year_yesterday)

# three_months_ago = datetime.now() -  relativedelta(months=2)
# three_months_ago_day = int(three_months_ago.strftime("%d"))
# three_months_ago_day = three_months_ago_day + 1
# #three_months_ago_day = "0" + str(three_months_ago_day)
# three_months_ago_day = str(three_months_ago_day)
# three_months_ago_month = three_months_ago.strftime("%m")
# three_months_ago_year = three_months_ago.strftime("20%y")
# print(three_months_ago_day, three_months_ago_month, three_months_ago_year)

In [4]:
s3 = boto3.client('s3')
# s3.download_file('cortex-modeling-andes-local', 'epiphany-cortex/TABLA_DIARIO/diario_leakage_2024-08-01_2024-09-29.csv', 'diario_leakage_2024-08-01_2024-09-29.csv')
# s3.download_file('cortex-modeling-andes-local', 'epiphany-cortex/TABLA_DIARIO/diario_leakage_{}-{}-{}_{}-{}-{}.csv'.format(three_months_ago_year, three_months_ago_month, three_months_ago_day, year_yesterday, month_yesterday, yesterday), 'diario_leakage_{}-{}-{}_{}-{}-{}.csv'.format(three_months_ago_year, three_months_ago_month, three_months_ago_day, year_yesterday, month_yesterday, yesterday))

In [5]:
import subprocess
from datetime import datetime, timedelta
# Configuración de fechas
today = datetime.now().date()
yesterday = today - timedelta(1)
three_months_ago = yesterday - timedelta(59)
# Generar el nombre del archivo basado en las fechas
file_name = f'diario_leakage_{three_months_ago}_{yesterday}.csv'
# Ruta en S3
s3_folder = 's3://hdi-sagemaker-project-co/data-science/nppm-latam/fraude/daily_claims_colombia/daily_claims_colombia_danios_test/'
# Comando para descargar usando AWS CLI
try:
    print(f"Descargando el archivo: {s3_folder}{file_name}")
    subprocess.run(['aws', 's3', 'cp', f'{s3_folder}{file_name}', './diario.csv'], check=True)
    print("Archivo descargado exitosamente como 'diario.csv'")
except subprocess.CalledProcessError as e:
    print(f"Error al descargar el archivo: {e}")

Descargando el archivo: s3://hdi-sagemaker-project-co/data-science/nppm-latam/fraude/daily_claims_colombia/daily_claims_colombia_danios_test/diario_leakage_2024-11-15_2025-01-13.csv


download: s3://hdi-sagemaker-project-co/data-science/nppm-latam/fraude/daily_claims_colombia/daily_claims_colombia_danios_test/diario_leakage_2024-11-15_2025-01-13.csv to ./diario.csv
Archivo descargado exitosamente como 'diario.csv'


In [6]:
data = pd.read_csv("diario.csv")

In [7]:
# # data = pd.read_csv("diario_leakage_2024-08-01_2024-09-29.csv")
# data = pd.read_csv('diario_leakage_{}-{}-{}_{}-{}-{}.csv'.format(three_months_ago_year, three_months_ago_month, three_months_ago_day, year_yesterday, month_yesterday, yesterday))
# data.head(2)

In [8]:
# data = pd.read_csv("s3://hdi-sagemaker-project-co/data-science/nppm-latam/fraude/daily_claims_colombia/daily_claims_colombia_danios_test/diario_leakage_2024-10-30_2024-12-17.csv")

In [9]:
#data.head(2)

In [10]:
data['fecha_proceso_date'] = pd.to_datetime(data['fecha_proceso'])

In [11]:
data.dtypes

numero_siniestro                            int64
poliza_asociada                             int64
placa_vehiculo                             object
cod_fasecolda                               int64
descripcion_riesgo                         object
valor_vehiculo                            float64
documento                                   int64
amparo                                      int64
descripcion_deducible                      object
valor_deducible                            object
descripcion_siniestro                      object
1.- Sección delantera                      object
2.- Lateral delantero izquierdo            object
3.- Lateral delantero derecho              object
4.- Lateral trasero izquierdo              object
5.- Lateral trasero derecho                object
6.- Sección posterior                      object
7.- Techo                                  object
codigo_marca                                int64
descripcion_marca                          object


In [12]:
data.head(2)

,numero_siniestro,poliza_asociada,placa_vehiculo,cod_fasecolda,descripcion_riesgo,valor_vehiculo,documento,amparo,descripcion_deducible,valor_deducible,...,codigo_clase,descripcion_clase,codigo_tipo_vehiculo,descripcion_tipo_vehiculo,anio_modelo,departamento,municipio,tipo_declarante,fecha_proceso,fecha_proceso_date
0,1118803,516197,FPR887,9201247,FPR887 - VOLKSWAGEN - JETTA [7] - TSI COMFORTL...,66200000.0,1,760,PPD - Pérdida Parcial por Daños,0% con un mínímo de 1 SMMLV,...,1,AUTOMOVILES,247,JETTA,2019,Bogotá D.C,"BOGOTÁ, D.C",Asegurado,2024-11-15,2024-11-15
1,1118808,94175852,REN631,601120,REN631 - AUDI - A4 - B8 1.8 TFSI ATT,43000000.0,3,760,Deducible Pérdida Parcial Daños,1 SMLV,...,1,AUTOMOVILES,120,AUDI-A4,2011,Bogotá D.C,"BOGOTÁ, D.C",Asegurado,2024-11-15,2024-11-15


In [13]:
#data = data[data['fecha_proceso_date'] >= '2024-02-12']
#data.shape[0]

In [14]:
def claims_keep_by_date(data):
    if datetime.today().weekday() == 0:
        return data[data['fecha_proceso_date'] >= pd.to_datetime((datetime.today().date() - timedelta(days=30)))]
    
    else:
        return data[data['fecha_proceso_date'] >= pd.to_datetime((datetime.today().date() - timedelta(days=80)))]

In [15]:
data = claims_keep_by_date(data)

In [16]:
data['fecha_proceso_date'].value_counts(dropna = False)

2025-01-13    107
2025-01-07    107
2024-12-02    106
2024-12-09     94
2024-12-23     89
2024-11-18     89
2025-01-02     89
2024-12-16     85
2024-11-25     83
2024-12-26     82
2024-12-11     78
2024-12-30     78
2025-01-08     76
2024-11-19     75
2024-12-03     74
2024-11-20     70
2024-12-17     68
2024-12-20     68
2024-11-21     64
2024-12-10     63
2024-12-04     62
2024-12-05     61
2024-11-26     61
2024-12-18     59
2024-12-12     59
2024-11-15     59
2024-11-22     58
2024-12-27     58
2024-11-29     57
2025-01-09     57
2025-01-10     57
2024-12-19     57
2024-12-13     56
2024-11-27     56
2024-12-06     52
2024-12-28     46
2024-11-28     44
2025-01-11     41
2025-01-03     40
2024-12-07     39
2024-12-24     37
2024-12-14     35
2024-11-30     34
2024-11-16     34
2024-12-21     30
2024-11-23     28
2024-12-31     24
2024-12-15     21
2024-11-17     20
2025-01-12     18
2025-01-06     15
2024-11-24     14
2024-12-22     13
2024-12-01     12
2024-12-08     11
2024-12-29

In [17]:
data.drop(columns = ['fecha_proceso_date'], inplace = True)

In [18]:
data.reset_index(drop = True, inplace = True)

In [19]:
data.head(2)

,numero_siniestro,poliza_asociada,placa_vehiculo,cod_fasecolda,descripcion_riesgo,valor_vehiculo,documento,amparo,descripcion_deducible,valor_deducible,...,descripcion_marca,codigo_clase,descripcion_clase,codigo_tipo_vehiculo,descripcion_tipo_vehiculo,anio_modelo,departamento,municipio,tipo_declarante,fecha_proceso
0,1118803,516197,FPR887,9201247,FPR887 - VOLKSWAGEN - JETTA [7] - TSI COMFORTL...,66200000.0,1,760,PPD - Pérdida Parcial por Daños,0% con un mínímo de 1 SMMLV,...,VOLKSWAGEN,1,AUTOMOVILES,247,JETTA,2019,Bogotá D.C,"BOGOTÁ, D.C",Asegurado,2024-11-15
1,1118808,94175852,REN631,601120,REN631 - AUDI - A4 - B8 1.8 TFSI ATT,43000000.0,3,760,Deducible Pérdida Parcial Daños,1 SMLV,...,AUDI,1,AUTOMOVILES,120,AUDI-A4,2011,Bogotá D.C,"BOGOTÁ, D.C",Asegurado,2024-11-15


In [20]:
numero_de_siniestro = data.numero_siniestro
valor_vehiculo = data.valor_vehiculo
descripcion = data.descripcion_siniestro
fecha = data.fecha_proceso
placa = data.placa_vehiculo

In [21]:
columns_to_keep = ['valor_vehiculo', 'valor_deducible',
                   'descripcion_siniestro', '1.- Sección delantera',
                   '2.- Lateral delantero izquierdo', '3.- Lateral delantero derecho',
                   '4.- Lateral trasero izquierdo', '5.- Lateral trasero derecho',
                   '6.- Sección posterior', '7.- Techo', 'codigo_marca', 'codigo_clase', 
                   'codigo_tipo_vehiculo', 'anio_modelo',
                   'municipio']
                  
data = data[columns_to_keep]

In [22]:
def normalize_text(value): 
    if pd.isna(value):
        return None

    # Lowercase, remove extra spaces, and standardize similar terms
    value = value.lower().strip()
    value = value.replace('í', 'i').replace(',', '.').replace('%', '').replace('$', '')
    value = value.strip()
    value = re.sub(r'\s+', ' ', value)  # Replace multiple spaces with a single space

    return value

# Apply the normalization function to the 'valor_deducible' column
data['valor_deducible_normalized'] = data['valor_deducible'].apply(normalize_text)

# Display unique normalized values for verification
normalized_unique_values = data['valor_deducible_normalized'].unique()
normalized_unique_values

array(['0 con un minimo de 1 smmlv', '1 smlv', '10 min 1 smlv',
       '0 con un minimo de 700.000', '2 smlv',
       '10 con un minimo de 1 smmlv', '10 min 4 smlv', '0',
       '20 con un minimo de 4 smmlv', '950.000',
       '0 con un minimo de 1.5 smmlv', '10 con un minimo de 3 smmlv',
       '20 con un minimo de 6 smmlv', '10 min 2 smlv',
       '10 con un minimo de 2 smmlv', '15 con un minimo de 10 smmlv',
       '10 min 1.2 smlv', '10 con un minimo de 1.5 smmlv',
       '10 min 5 smlv', '20 con un minimo de 0 smmlv', '3 smlv',
       '30 con un minimo de 2 smmlv', '10 min 3 smlv',
       '10 con un minimo de 4 smmlv', '20 con un minimo de 3 smmlv',
       '1.1 smlv', '15 con un minimo de 4 smmlv', None,
       '10 con un minimo de 0 smmlv', 'sin deducible', '700.000',
       '30 min 3 smlv', '0 con un minimo de 1.200.000',
       '15 con un minimo de 0 smmlv', '20 min 2 smlv',
       '20 con un minimo de 2 smmlv'], dtype=object)

In [23]:
columns_to_encoder = ['1.- Sección delantera',
       '2.- Lateral delantero izquierdo', '3.- Lateral delantero derecho',
       '4.- Lateral trasero izquierdo', '5.- Lateral trasero derecho',
       '6.- Sección posterior', '7.- Techo']

In [24]:
for c in columns_to_encoder:
    data[c] = data[c].apply(lambda x: 1 if x == "X" else 0)

In [25]:
data['num_partes_afectadas'] = data[columns_to_encoder].sum(axis=1)

In [26]:
data = data.drop(columns = ['valor_deducible'])

In [27]:
data.rename(columns = {'valor_deducible_normalized': 'valor_deducible'}, inplace = True)

In [28]:
data.dtypes

valor_vehiculo                     float64
descripcion_siniestro               object
1.- Sección delantera                int64
2.- Lateral delantero izquierdo      int64
3.- Lateral delantero derecho        int64
4.- Lateral trasero izquierdo        int64
5.- Lateral trasero derecho          int64
6.- Sección posterior                int64
7.- Techo                            int64
codigo_marca                         int64
codigo_clase                         int64
codigo_tipo_vehiculo                 int64
anio_modelo                          int64
municipio                           object
valor_deducible                     object
num_partes_afectadas                 int64
dtype: object

In [29]:
data.head(2)

,valor_vehiculo,descripcion_siniestro,1.- Sección delantera,2.- Lateral delantero izquierdo,3.- Lateral delantero derecho,4.- Lateral trasero izquierdo,5.- Lateral trasero derecho,6.- Sección posterior,7.- Techo,codigo_marca,codigo_clase,codigo_tipo_vehiculo,anio_modelo,municipio,valor_deducible,num_partes_afectadas
0,66200000.0,Iba conduciendo por la calle 116 entre la carr...,1,0,0,0,0,0,0,92,1,247,2019,"BOGOTÁ, D.C",0 con un minimo de 1 smmlv,1
1,43000000.0,VENIAMOS POR LA BOYACA SE VA ATOMAR OREJA PARA...,1,1,0,0,0,0,0,6,1,120,2011,"BOGOTÁ, D.C",1 smlv,2


In [30]:
data.shape[0]

3106

In [31]:
from typing import List, Dict

In [32]:
optuned_model = joblib.load('co_totalloss.joblib')
type(optuned_model)

/home/ec2-user/SageMaker/totalloss_Liberty/totalloss/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


nppm_latam_optuner.optuner.OptunedModel

In [33]:
optuned_model

In [34]:
import sklearn
print("Versión actual de scikit-learn:", sklearn.__version__)

Versión actual de scikit-learn: 1.1.2


In [35]:
import joblib
print("Versión actual de joblib:", joblib.__version__)

Versión actual de joblib: 1.2.0


In [36]:
#probas = optuned_model.predict_proba(data)[:,1]
probas = optuned_model.predict_proba(data)

data.shape, probas.shape

((3106, 16), (3106, 2))

In [37]:
data.head(2)

,valor_vehiculo,descripcion_siniestro,1.- Sección delantera,2.- Lateral delantero izquierdo,3.- Lateral delantero derecho,4.- Lateral trasero izquierdo,5.- Lateral trasero derecho,6.- Sección posterior,7.- Techo,codigo_marca,codigo_clase,codigo_tipo_vehiculo,anio_modelo,municipio,valor_deducible,num_partes_afectadas
0,66200000.0,Iba conduciendo por la calle 116 entre la carr...,1,0,0,0,0,0,0,92,1,247,2019,"BOGOTÁ, D.C",0 con un minimo de 1 smmlv,1
1,43000000.0,VENIAMOS POR LA BOYACA SE VA ATOMAR OREJA PARA...,1,1,0,0,0,0,0,6,1,120,2011,"BOGOTÁ, D.C",1 smlv,2


In [38]:
data['probabilidades'] = probas[:,1]
data['numero_siniestro'] = numero_de_siniestro
data['fecha_reporte'] = fecha
data['placa'] = placa
data['perdida_total'] = np.where(data['probabilidades'] >= 0.30, 'total', 'parcial')

In [39]:
data.head(2)

,valor_vehiculo,descripcion_siniestro,1.- Sección delantera,2.- Lateral delantero izquierdo,3.- Lateral delantero derecho,4.- Lateral trasero izquierdo,5.- Lateral trasero derecho,6.- Sección posterior,7.- Techo,codigo_marca,...,codigo_tipo_vehiculo,anio_modelo,municipio,valor_deducible,num_partes_afectadas,probabilidades,numero_siniestro,fecha_reporte,placa,perdida_total
0,66200000.0,Iba conduciendo por la calle 116 entre la carr...,1,0,0,0,0,0,0,92,...,247,2019,"BOGOTÁ, D.C",0 con un minimo de 1 smmlv,1,0.049576,1118803,2024-11-15,FPR887,parcial
1,43000000.0,VENIAMOS POR LA BOYACA SE VA ATOMAR OREJA PARA...,1,1,0,0,0,0,0,6,...,120,2011,"BOGOTÁ, D.C",1 smlv,2,0.105358,1118808,2024-11-15,REN631,parcial


In [40]:
data = data.sort_values(by=['probabilidades'], ascending = False)

In [41]:
data = data[['numero_siniestro', 'perdida_total', 'valor_vehiculo', 'fecha_reporte', 'placa', 'descripcion_siniestro', '1.- Sección delantera', '2.- Lateral delantero izquierdo', '3.- Lateral delantero derecho', '4.- Lateral trasero izquierdo', '5.- Lateral trasero derecho', '6.- Sección posterior', '7.- Techo', 'codigo_marca', 'codigo_clase', 'codigo_tipo_vehiculo', 'anio_modelo', 'municipio', 'valor_deducible', 'num_partes_afectadas', 'probabilidades']]
data.head(2)

,numero_siniestro,perdida_total,valor_vehiculo,fecha_reporte,placa,descripcion_siniestro,1.- Sección delantera,2.- Lateral delantero izquierdo,3.- Lateral delantero derecho,4.- Lateral trasero izquierdo,...,6.- Sección posterior,7.- Techo,codigo_marca,codigo_clase,codigo_tipo_vehiculo,anio_modelo,municipio,valor_deducible,num_partes_afectadas,probabilidades
2430,1125413,total,30200000.0,2024-12-30,DQX940,TRAYECTO A LERIDA TOLIMA APARENTEMENTE A UN MI...,1,1,1,1,...,1,1,16,1,308,2018,ARMENIA,1 smlv,7,0.949459
822,1121264,total,43700000.0,2024-11-30,UEV768,Me desplazaba por vía de Manizales hacia Santa...,1,1,1,1,...,1,1,80,1,176,2017,MANIZALES,1 smlv,7,0.943600


In [42]:
#data.to_excel("probabilitiestotaloss.csv", index = False)
#get the day/month and year of month
import datetime
time = datetime.datetime.now()
d = time.strftime("%d")
m = time.strftime("%m")
y = time.strftime("%y")

data.to_csv("probabilitiestotaloss_{}_{}_{}.csv".format(d,m,y), index = False)

#s3.upload_file('probabilitiestotaloss_{}_{}_{}.csv'.format(d,m,y), 'cortex-intl-andes-nppm-latam', 'totallossco/predictions/probabilitiestotaloss_{}_{}_{}.csv'.format(d,m,y))

In [43]:
data2 = pd.DataFrame(data)

In [44]:
columns_to_keep=['numero_siniestro', 'perdida_total', 'valor_vehiculo', 'descripcion_siniestro', 'fecha_reporte', 'placa', '1.- Sección delantera', '2.- Lateral delantero izquierdo', '3.- Lateral delantero derecho', '4.- Lateral trasero izquierdo', '5.- Lateral trasero derecho', '6.- Sección posterior', '7.- Techo']

data2 = data2[columns_to_keep]

In [45]:
# data2.to_excel("perdidas_totales_{}_{}_{}.xlsx".format(d,m,y), index = False)
# s3.upload_file('perdidas_totales_{}_{}_{}.xlsx'.format(d,m,y), 'cortex-intl-andes-nppm-latam', 'totallossco/predictions/perdidas_totales_{}_{}_{}.xlsx'.format(d,m,y))

In [46]:
# Guardar el archivo localmente como Excel
output_file = "perdidas_totales_{}_{}_{}.xlsx".format(d, m, y)
data2.to_excel(output_file, index=False)
# Nueva configuración para la ruta de subida en S3
bucket_name = 'hdi-sagemaker-project-co'
prefix = 'data-science/nppm-latam/totallossco/predictions/'
# Subir el archivo a la nueva ubicación en S3
s3.upload_file(
   Filename=output_file,
   Bucket=bucket_name,
   Key=prefix + output_file
)

In [47]:
print("Archivo subido correctamente a hdi-sagemaker-project-co/data-science/nppm-latam/totallossco/predictions/")

Archivo subido correctamente a hdi-sagemaker-project-co/data-science/nppm-latam/totallossco/predictions/


In [48]:
#############